In [ ]:
%%shell

pip install cython
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-detc2c3s
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-detc2c3s
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264358 sha256=65418eaff059fce0a38deb257eea40d3b226095cb452f9e929c1a52e20fe445f
  Stored in directory: /tmp/pip-ephem-wheel-cache-g08bb288/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


# Penn-Fundan 다운로드

In [ ]:
%%shell

# download the Penn-Fudan dataset
wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip

# extract it in the current folder
unzip PennFudanPed.zip

--2022-03-23 06:34:31--  https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
Resolving www.cis.upenn.edu (www.cis.upenn.edu)... 158.130.69.163, 2607:f470:8:64:5ea5::d
Connecting to www.cis.upenn.edu (www.cis.upenn.edu)|158.130.69.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53723336 (51M) [application/zip]
Saving to: ‘PennFudanPed.zip.1’

PennFudanPed.zip.1  100%[===================>]  51.23M  30.5MB/s    in 1.7s    

2022-03-23 06:34:33 (30.5 MB/s) - ‘PennFudanPed.zip.1’ saved [53723336/53723336]

Archive:  PennFudanPed.zip
replace PennFudanPed/added-object-list.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from PIL import Image
Image.open('PennFudanPed/PNGImages/FudanPed00001.png')

In [ ]:
mask = Image.open("PennFudanPed/PedMasks/FudanPed00001_mask.png")

In [ ]:
mask.putpalette([
                 0,0,0, # black background
                 255, 0, 0, # index 1은 red
                 255, 255, 0, # index 2는 yellow
                 255, 153, 0, # index 3은 오렌지 
])
mask

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # 모든 이미지와 피일 읽고 정렬
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # 이미지와 마스크 읽기
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # img만 RGB로 변환
        mask = Image.open(mask_path)

        # numpy 배열을 PIL이미지로 변환
        mask = np.array(mask)
        # 인스턴스들은 다른색으로 코딩되어 있음
        obj_ids = np.unique(mask)
        # 첫번째 id는 배경이라 제거
        obj_ids = obj_ids[1:]

        # 컬러 인코딩된 마스크를 바이너리 마스크 세트로 나눔
        masks = mask == obj_ids[:, None, None]

        # 각 마스크의 바운딩 박스 좌표를 얻음
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # 모든 것을 torch tensor 좌표로 바꿈
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 객체 종류는 한 종류만 존재(여기에서는 사람)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 모든 인스턴스는 군중 상태가 아님을 가정
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
dataset = PennFudanDataset('PennFudanPed/')
dataset[0]

# 미리 학습된 모델로부터 미세 조정

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# COCO로 미리 학습된 모델 읽기
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 분류기를 새로운 것으로 교체 (배경 + 사람) 2개의 클래스
num_classes = 2

# 분류기에서 사용할 입력 특징의 차원 정보 얻기
in_features = model.roi_heads.box_predictor.cls_score.in_features

# 미리 학습된 모델의 머리 부분을 새로운 것으로 교체
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# 다른 백본을 추가하도록 모델을 수정

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# 분류 목적으로 미리 학습된 모델을 로드하고 특징들만 리턴
backbone = torchvision.models.mobilenet_v2(pretrained=True).features

# Faster RCNN은 백본의 출력 채널 수를 알아야 함
# mobilenetV2의 경우 1280이므로 여기에 추가
backbone.out_channels = 1280

# RPN이 5개의 서로 다른 크기와 3개의 다른 측면 비율을 가진 5 x 3개의 앵커를 공간 위치마다 생성
# 각 특징 맵이 잠재적으로 다른 사이즈와 측면 비율을 가질 수 있기 때문에 Tuple[Tuple[int]] 타입을 가지도록 함

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios =((0.5,1.0,2.0),))

# 관심 영역의 자르기 및 재할당 후 자르기 크기를 수행하는데 사용할 피처맵 정의
# 백본이 텐서를 리턴할 때, featmap_names는 [0]이 될 것이라 예상
# 일반적으로 백본을 OrderedDict[Tensor] 타입을 리턴해야 함
# 특징맵에서 사용할 featmap_names 값을 정할 수 있음
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# 조각들을 Faster RCNN 모델로 합침
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)


# PennFudan 데이터셋을 위한 인스턴스 분할 모델

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # COCO에서 미리 학습된 인스턴스 분할 모델 읽어오기
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained = True)

    # 분류를 위한 입력 특징 차원 얻기
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 미리 학습된 헤더를 새로운 것으로 바꿈
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 마스크 분류기를 위한 입력 특징 차원 얻기
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    # 마스크 예측기를 새로운 것으로 바꿈
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

    return model

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

# 모든 것을 하나로 합치기

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
  transforms = []
  transforms.append(T.ToTensor())
  if train:
      transforms.append(T.RandomHorizontalFlip(0.5))
  return T.Compose(transforms)

# forward() 메소드 테스트하기

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size = 2, shuffle = True, num_workers = 4,
    collate_fn=utils.collate_fn)

# 학습 시
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)

# 추론 시
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)

In [ ]:
from engine import train_one_epoch, evaluate
import utils

def main():
  # 학습을 GPU로 진행하되 없으면 CPU로 진행
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")

  # 배경과 사람 2개의 클래스
  num_classes = 2

  # 데이터셋 정의된 변환
  dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
  dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

  # train과 test셋 나누기(전체 50개 테스트, 나머지 학습)
  indices = torch.randperm(len(dataset)).tolist()
  dataset = torch.utils.data.Subset(dataset, indices[:-50])
  dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

  # 데이터 로더를 학습용과 검증용으로 정의
  data_loader = torch.utils.data.DataLoader(
      dataset, batch_size = 2, shuffle = True, num_workers = 4,
      collate_fn = utils.collate_fn
  )
  data_loader_test = torch.utils.data.DataLoader(
      dataset_test, batch_size = 1, shuffle = False, num_workers = 4,
      collate_fn = utils.collate_fn
  )

  # 도움 함수를 이용해 모델 가져오기
  model = get_model_instance_segmentation(num_classes)

  # 모델을 GPU나 CPU로 옮김
  model.to(device)

  # 옵티마이저를 만듦
  params = [p for p in model.parameters() if p.requires_grad]
  optimizer = torch.optim.SGD(params, lr = 0.005,
                              momentum = 0.9, weight_decay = 0.0005)
  
  # 학습을 스케줄러를 만듦
  lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)


  # 10 에포크만 학습
  num_epochs = 10


  for epoch in range(num_epochs):
    # 1 에포크학습 10회마다 출력
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq = 10)
    # 학습률 업데이트
    lr_scheduler.step()
    # 테스트 데이터셋 평가
    evaluate(model, data_loader_test, device = device)

  print("That's it!")


In [ ]:
main()